<a href="https://colab.research.google.com/github/pejmanrasti/MiFoBio2021/blob/main/1_Mnist_DL_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
%matplotlib inline

TensorFlow 1.x selected.


# **Make your fist experience with Tensorflow-Keras**
Our goal is to construct and train an artificial neural network on thousands of images of handwritten digits so that it may successfully identify others when presented. The data that will be incorporated is the MNIST database which contains 60,000 images for training and 10,000 test images.

## Loading Training and Validation Data

The MNIST dataset is conveniently bundled within Keras, and we can easily analyze some of its features in Python.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf


In [ ]:
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
# load (downloaded if needed) the MNIST dataset
(X_train, y_train), (X_val, y_val) = mnist.load_data()

# **Importing necessary Libraries**

In [ ]:
! pip install TensorBoardColab
from tensorflow.keras.models import Sequential # Model type to be used
from tensorflow.keras.layers import Dense, Activation, Dropout# Make Fully connected (FC) layers
from tensorflow.keras.utils import to_categorical # NumPy related tools
from tensorflow.keras.callbacks import TensorBoard  #Visulization of Accuracy and loss
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

import numpy as np                   # advanced math library
import matplotlib.pyplot as plt      # MATLAB like plotting routines
import random                        # for generating random numbers


Visualization of some input images

In [ ]:
plt.rcParams['figure.figsize'] = (9,9) # Make the figures a bit bigger

for i in range(9):
    plt.subplot(3,3,i+1)
    num = random.randint(0, len(X_train))
    plt.imshow(X_train[num], cmap='gray', interpolation='none')
    plt.title("Class {}".format(y_train[num]))
    
plt.tight_layout()

## Formatting the input data layer

Instead of a 28 x 28 matrix, we build our network to accept a 784-length vector. Each image needs to be then reshaped (or flattened) into a vector. 

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 784).astype('float32')
X_val = X_val.reshape(X_val.shape[0], 784).astype('float32')

# normalize inputs from 0-255 to 0-1
X_train /= 255
X_val /= 255

We then modify our classes (unique digits) to be in the one-hot format, i.e.

In [ ]:
# one hot encode outputs
Y_train = to_categorical(y_train)
Y_val = to_categorical(y_val)

# Building the simplest fully connected network (FCN) with just one layer

In [ ]:
# The Sequential model is a linear stack of layers and is very common.
model = Sequential([
    Dense(10), # It is the output layer and should be equal to the number of desired classes (10 in this case).
    Activation('softmax'),
])


## Compiling the model

Keras is built on top of TensorFlow. It allows you to define a *computation graph* in Python, which then compiles and runs efficiently on the CPU or GPU without the overhead of the Python interpreter.

When compiling a model, Keras asks you to specify your **loss function** and your **optimizer**. The loss function we'll use here is called *categorical cross-entropy*, and is a loss function well-suited to comparing two probability distributions.

Our predictions are probability distributions across the ten different digits (e.g. "we're 80% confident this image is a 3, 10% sure it's an 8, 5% it's a 2, etc."), and the target is a probability distribution with 100% for the correct category, and 0 for everything else. The cross-entropy is a measure of how different your predicted distribution is from the target distribution. [More detail at Wikipedia](https://en.wikipedia.org/wiki/Cross_entropy)

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

# **TensorFlow's visualization toolkit**

TensorBoard provides the visualization and tooling needed for machine learning experimentation:
*   Tracking and visualizing metrics such as loss and accuracy
*   Visualizing the model graph (ops and layers)
*   Viewing histograms of weights, biases, or other tensors as they change over time
*   Projecting embeddings to a lower dimensional space
*   Displaying images, text, and audio data
*   Profiling TensorFlow programs
*   And much more

After running the following block, you should click on the generated lik.



In [ ]:
tbc=TensorBoardColab()

## Train the model!

In [ ]:
model.fit(X_train, Y_train, 
          validation_data=(X_val, Y_val),
          epochs=20, batch_size=32,
          verbose=1,
          callbacks=[TensorBoardColabCallback(tbc)])

## Evaluate Model's Accuracy on Test Data
Your test data **Must** be different from the validation data, but in this example, we will use the validation data as the test data as well.

In [ ]:
score = model.evaluate(X_val, Y_val)
print('Test score:', score[0])
print('Test accuracy:', score[1])

### Inspecting the output

It's always a good idea to inspect the output and make sure everything looks sane. Here we'll look at some examples it gets right, and some examples it gets wrong.

In [ ]:
# The predict_classes function outputs the highest probability class
# according to the trained classifier for each input example.
predicted_classes = model.predict_classes(X_val)

# Check which items we got right / wrong
correct_indices = np.nonzero(predicted_classes == y_val)[0]

incorrect_indices = np.nonzero(predicted_classes != y_val)[0]

In [ ]:
plt.figure()
for i, correct in enumerate(correct_indices[:6]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_val[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], y_val[correct]))
    
plt.tight_layout()
    

In [ ]:
plt.figure()
for i, incorrect in enumerate(incorrect_indices[:6]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_val[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], y_val[incorrect]))
    
plt.tight_layout()

# **Let's go Deeper**
We will add four more layers to our model. We use Droupout in our model to reduce overfitting

In [ ]:
# Dropout helps protect the model from memorizing or "overfitting" the training data.
Pkeep=0.25
modelDeepFC = Sequential([
    Dense(200, input_shape=(784,)),
    Activation('relu'),
    Dropout(Pkeep),
    Dense(100, input_shape=(200,)),
    Activation('relu'),
    Dropout(Pkeep),
    Dense(60, input_shape=(100,)),
    Activation('relu'),
    Dropout(Pkeep),
    Dense(30, input_shape=(60,)),
    Activation('relu'),
    Dropout(Pkeep),
    Dense(10),
    Activation('softmax'),
])

In [ ]:
tbcDeepFC=TensorBoardColab()

In [ ]:
modelDeepFC.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

modelDeepFC.fit(X_train, Y_train, 
          validation_data=(X_val, Y_val),
          epochs=20, batch_size=32,
          verbose=1,
          callbacks=[TensorBoardColabCallback(tbcDeepFC)])

The performance of the last model with more layers showed a better performance compare with our first model.

# **Let's go furter with introducing CNN**
Before, we built a network that accepts the normalized pixel values of each value and operates soley on those values. What if we could instead feed different features (e.g. curvature, edges) of each image into a network, and have the network learn which features are important for classifying an image?

This possible through convolution! Convolution applies kernels (filters) that traverse through each image and generate feature maps.

In [ ]:
# import some additional tools

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten

In [ ]:
# Reload the MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# Again, do some formatting
# Except we do not flatten each image into a 784-length vector because we want to perform convolutions first

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') #add an additional dimension to represent the single-channel
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

X_train /= 255                              # normalize each value for each pixel for the entire vector for each input
X_test /= 255

print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

In [ ]:
# one hot encode outputs
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

In [ ]:
modelCNN = Sequential([
    
    # Convolution Layer 1
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)), # 32 different 3x3 kernels -- so 32 feature maps
    MaxPooling2D(pool_size=(2, 2)), # Pool the max values over a 2x2 kernel

    # Convolution Layer 2
    Conv2D(64, (3, 3), activation='relu'), # 64 different 3x3 kernels 
    MaxPooling2D(pool_size=(2, 2)),

    # Convolution Layer 3
    Conv2D(128, (3, 3), activation='relu'), # 128 different 3x3 kernels

    Flatten(), # Flatten final 7x7x128 output matrix into a 1024-length vector 

    # Fully Connected Layer 4
    Dense(512), # 512 FCN nodes
    Activation('relu'),
    Dropout(0.2),
    Dense(10),
    Activation('softmax'),
])
modelCNN.summary()

In [ ]:
tbcCNN=TensorBoardColab()

In [ ]:
modelCNN.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

modelCNN.fit(X_train, Y_train, 
          validation_data=(X_test, Y_test),
          epochs=20, batch_size=32,
          verbose=1,
          callbacks=[TensorBoardColabCallback(tbcCNN)])

**Evaluation and Prediction**

We can use our model to make a prediction on new images.

In [ ]:
modelCNN.evaluate(X_test,Y_test) #Evaluation of the model on the test dataset

In [ ]:
modelCNN.predict_classes(X_test) # Prediction of classes

In [ ]:
# Prediction of classes of a single image
img=X_test[1000,:,:,:]
img = np.array(img).reshape(-1, 28, 28, 1)
output = modelCNN.predict_classes(img)
print('The predicted label is: ',output[0])
print('The real label is: ',y_test[1000])